# Problem Sheet 8

## Convexity and Stochastic Gradient Descent (SGD) method

## Task (a)
Consider $\boldsymbol\theta=(\theta_1,\theta_2) \in \mathbb{R}^2$ and the function
$$
L(\boldsymbol\theta) = \theta_1^2 + (\theta_1 - \theta_2)^2.
$$
- Prove that $L(\boldsymbol\theta)$ is convex on $\mathbb{R}^2$.

_Hint: use Lemma 4.24 and that_ $2ab \le a^2+b^2$ _for any_ $a,b \in \mathbb{R}$.

## Solution
$$
L(\boldsymbol\theta) = 2\theta_1^2 - 2 \theta_1 \theta_2 + \theta_2^2,
$$
hence
$$
\nabla L(\boldsymbol\theta) = \begin{bmatrix}4 \theta_1 - 2 \theta_2 \\ -2 \theta_1 + 2 \theta_2\end{bmatrix}.
$$
Using Lemma 4.24, to check convexity we can check nonnegativity of the following inner product:
\begin{align*}
\langle \nabla L(\boldsymbol\eta) - \nabla L(\boldsymbol\theta), \boldsymbol\eta - \boldsymbol\theta \rangle & = \left\langle\begin{bmatrix}4 (\eta_1-\theta_1) - 2 (\eta_2-\theta_2) \\ -2 (\eta_1-\theta_1) + 2 (\eta_2-\theta_2)\end{bmatrix}, \begin{bmatrix}\eta_1 - \theta_1 \\ \eta_2 - \theta_2\end{bmatrix} \right\rangle \\
& = 4 (\eta_1-\theta_1)^2 - 4 \underbrace{(\eta_1 - \theta_1)}_a \underbrace{(\eta_2 - \theta_2)}_b + 2 (\eta_2 - \theta_2)^2 \\
& \ge 4 (\eta_1-\theta_1)^2 - 2 (\eta_1-\theta_1)^2 - 2 (\eta_2-\theta_2)^2 + 2 (\eta_2-\theta_2)^2 \\
& = 2 (\eta_1 - \theta_1)^2 \ge 0.
\end{align*}

## Task (b) (Warm-up)
Consider the same function as in Task (a).
- Find $m \in \mathbb{N}$, $\mathbf{x}_i \in \mathbb{R}^2$ and $y_i \in \mathbb{R}$, $i=1,\ldots,m$, such that $L(\boldsymbol\theta)$ is equal to the loss $L(\boldsymbol\theta) = \frac{1}{m} \sum_{i=1}^{m}(\langle \boldsymbol\theta, \mathbf{x}_i \rangle - y_i)^2$ of the linear regression problem.

## Solution:
We can rewrite
\begin{align*}
L(\boldsymbol\theta) & = \langle \boldsymbol\theta, [1,0] \rangle^2 + \langle \boldsymbol\theta, [1,-1] \rangle^2 \\
& = \frac{1}{2}\left(\langle \boldsymbol\theta, [\sqrt{2},0] \rangle^2 + \langle \boldsymbol\theta, [\sqrt{2},-\sqrt{2}] \rangle^2 \right)
\end{align*}

So $m=2$, $y_1=y_2=0$, and 
$$
\mathbf{x}_1 = \begin{bmatrix}\sqrt{2} \\ 0\end{bmatrix}, \quad \mathbf{x}_2 = \begin{bmatrix}\sqrt{2} \\ -\sqrt{2}\end{bmatrix} \quad \Rightarrow \quad X = \sqrt{2} \begin{bmatrix}1 & 1 \\ 0 & -1\end{bmatrix}.
$$


## Task (c)
Consider the same function as in Task (a) and (b).
- Prove that $L(\boldsymbol\theta)$ is $\lambda$-strongly convex on $\mathbb{R}^2$ and suggest a value of $\lambda>0$.
 
_Hint: use Task (b) and Theorem 4.32._

## Solution:
Firstly, we assemble
$$
A = \frac{2}{m} XX^\top = 2 \begin{bmatrix}1 & 1 \\ 0 & -1\end{bmatrix} \begin{bmatrix}1 & 0 \\ 1 & -1\end{bmatrix} = \begin{bmatrix}4 & -2 \\ -2 & 2 \end{bmatrix}.
$$
By Theorem 4.32, $\lambda = \lambda_{\min}(A) = 3 - \sqrt{5} \approx 0.76$.

---

## Task 1: SGD for polynomial regression

- Write a Python function `sgd_poly(V, Y, theta0, t0, iters=10000)` which implements the Stochastic Gradient Descent method for the polynomial regression problem considered in Problem Sheet 7, taking 
$$
\mathbf{v}_k = \nabla_{\boldsymbol\theta} \tilde \ell_{x_i,y_i}(\boldsymbol\theta_k) = 2 \mathbf{v}_{i}^\top (\mathbf{v}_{i} \boldsymbol\theta - y_i),
$$
where $i=0,\ldots,m-1$ is sampled uniformly at random in each iteration, and $V \in \mathbb{R}^{m \times (d+1)}$ is as previously the Vandermonde matrix, and $\mathbf{v}_i \in \mathbb{R}^{n+1}$ is the $i$-th row of $V$. Once again, the code to compute $V$ and `Y`$=[y_1,\ldots,y_m]$ is as follows:

In [1]:
import numpy as np
from matplotlib import pyplot as plt    

def TrueSample():
    x = np.random.uniform(-1,1)
    y = x - x**3 + 0.1*np.random.randn()
    return x,y

def features(x,n):
    powers = np.arange(n+1)               # [0,1,2,...,n]
    powers = np.reshape(powers, (1, -1))  # Make it explicitly a row vector
    x = np.reshape(x, (-1, 1))            # Make it explicitly a column vector
    return x**powers                      # Python automatically broadcasts the vectors to each others' shapes 
                                          # and takes the power between the resulting matrices elementwise

Nsamples = 30
Y = np.zeros(Nsamples)
X = np.zeros(Nsamples)
for i in range(Y.size):
    X[i],Y[i] = TrueSample()

n = 3
V = features(X,n)

The other inputs to `sgd_poly` are the initial guess `theta0`$=\boldsymbol\theta_0$, the initial learning rate `t0`$=t_0$, which defines the learning rate $t_k = t_0/(k+1)$ in the $k$-th iteration, and the fixed number of iterations `iters`. The function `sgd_poly` should compute and return both `theta` **and** the loss `L(theta, V, Y)` as defined in Task 1 of Problem Sheet 7 at the final iteration.

_Hint: you may find the_ [np.random.randint](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html) _function useful._

## Solution:

In [2]:
def L(theta, V, Y):
    return np.linalg.norm(V @ theta - Y)**2 / Y.size

def sgd_poly(V, Y, theta0, t0, iters=10000):
    theta = theta0
    for k in range(iters):
        i = np.random.randint(low=0, high=V.shape[0])
        vk = 2 * V[i].T * (V[i] @ theta - Y[i])   # Note the (...) is a number
        tk = t0/(k+1)
        theta = theta - tk * vk    
    return theta, L(theta, V, Y)

## Task 2

- Write a Python code that applies `sgd_poly` to compute $\boldsymbol\theta_k \approx \boldsymbol\theta^* = \arg\min L(\boldsymbol\theta)$ for $L(\boldsymbol\theta)$ defined in Task 1, starting from $\boldsymbol\theta_0 = \mathbf{0}$, using default `iters`, and $t_0$ which you should select from the range $1/4,1/2,1,2,4$ such that the algorithm produces the smallest loss on average over a few (up to 10) restarts. What is this $t_0$ and the corresponding loss?

## Solution:

In [3]:
nrestarts = 10
for t0 in [1/4,1/2,1,2,4]:
    Loss = np.zeros(nrestarts)
    for i in range(nrestarts):
        theta, Loss[i] = sgd_poly(V, Y, np.zeros(n+1), t0)
    print(f"t0 = %g, avg. final loss = %3.3e" % (t0, np.mean(Loss)))

t0 = 0.25, avg. final loss = 3.168e-02
t0 = 0.5, avg. final loss = 2.089e-02
t0 = 1, avg. final loss = 1.533e-02
t0 = 2, avg. final loss = 1.135e-02
t0 = 4, avg. final loss = 1.393e+00


$t_0=1$ or $2$ depending on the particular realisation of data. Increasing `Nsamples` and `nrestarts` can give a more reproducible estimate.